### PSU DS 200  Fall 2019

#### Project Utility Jupyter Notebook 1
This Jupyter Notebook is intended to be used to construct Project Deliverables for the Mini-project
of DS 200. 
#### Acknowledgement: The datascience module is developed by University of California Berkeley for its data 8 course.


In [1]:
!pip install datascience

     |████████████████████████████████| 51kB 99kB/s  eta 0:00:01
     |████████████████████████████████| 92kB 280kB/s eta 0:00:01
     |████████████████████████████████| 13.0MB 18kB/s  eta 0:00:01    |██████████████████▎             | 7.4MB 2.6MB/s eta 0:00:03     |████████████████████████▌       | 10.0MB 959kB/s eta 0:00:04
     |████████████████████████████████| 215kB 9.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/65/e0/eb35e762802015cab1ccee04e8a277b03f1d8e53da3ec3106882ec42558b/Jinja2-2.10.3-py2.py3-none-any.whl
     |████████████████████████████████| 92kB 602kB/s  eta 0:00:01
  Created wheel for datascience: filename=datascience-0.15.3-cp35-none-any.whl size=44582 sha256=b39bdbc015d5c37c2006b29eea70825f68d548a1fdb537a6490f9ea59279f37e
  Stored in directory: /home/nbuser/.cache/pip/wheels/b8/37/0a/80274866028f6485c5957f0e1acf8e2b755fbe9dd0fd4ad275
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=19851 sha256=f9e7de27b985ed77163

In [2]:
import datascience
import numpy as np
import graphviz

from datascience import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 


from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.base import ClassifierMixin
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

import pandas as pd
import numpy as np

from sklearn.externals import joblib

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [7]:
t1 = Table.read_table("LabelledTweets.csv", sep =',')
t1.show(10)

FileNotFoundError: File b'LabelledTweets.csv' does not exist

### The code below extract the "class label" from the CSV file to find out the total number of positive/supportive vs negative/non-supportive tweets in the labelled data set.
### If you use Climage Change tweets, the column label for the class label is 'Support'.
### If you use Airline Sentiment tweets, the column label for the class label is 'airline_sentiment' (as shown in Lab9)

In [ ]:
t1_positive = t1.where('Support', are.equal_to(1))
t1_negative = t1.where('Support', are.equal_to(0))
t1_positive

In [ ]:
t1_negative

In [ ]:
data = t1_positive.append(t1_negative)

In [ ]:
data

In [ ]:
# Use the data for constructing a Decision Tree Stance Classifier
data_tagged_X= list(data['Text'])
data_tagged_Y= list(data['Support'])

print('tagged data input size', len(data_tagged_X))
print('tagged data target prediction size', len(data_tagged_Y))

### Stratified KFold Cross Validation
StratifiedKFold is a function in sklearn under the submodule model_selection.
#### Notice: In the beginning of this notebook, we import StratifiedKFold using the following python code:
    from sklearn.model_selection import StratifiedKFold
    
The n_splits parameter of StratifiedKFold indicates how many folds to use.  For example, 
n_splits=5 means Stratified 5-fold cross validation.


In [ ]:
# Use Stratified Kfold Cross Validation so that
#   each fold contains the same ratio of positive/negative instances
k = ???
skf = StratifiedKFold(n_splits= k , random_state=1, shuffle= True)

In [ ]:
### Set the range of max_depth for finding the best max_depth for this problem
low_max_depth = ???
high_max_depth = ???
training_performance_table = np.empty( [high_max_depth+1, 10, 5] )
testing_performance_table = np.empty( [high_max_depth+1, 10, 5] )

In [ ]:
for depth in range(low_max_depth, high_max_depth+1):

    fold = 1

    for train_index, test_index in skf.split(data_tagged_X, data_tagged_Y):
        print("Fold Number:", fold)
     #   print("Training Data Index:", train_index)
        print("Testing Data Index:", test_index)  
    
        x_train= list(data.take(train_index)['Text'])     
    #    print("Training Data:", x_train)
        y_train= list(data.take(train_index)['Support'])
    #    print("Training Data Target Output:", y_train)
        x_test= list(data.take(test_index)['Text'])
    #    print("Testing Data:", x_test)
        y_test= list(data.take(test_index)['Support'])
        print("Testing Data Target Output:", y_test)
    
        count_vect = CountVectorizer(token_pattern='((?:([@#]|[0-9]|[a-z]|[A-Z])+))', analyzer= 'word', min_df=2)
        X_word_vect = count_vect.fit_transform(x_train)
    
        clf = tree.DecisionTreeClassifier(criterion='entropy', random_state = 100, max_depth=depth, \
                                  min_samples_leaf =2)
    
        clf.fit(X_word_vect, y_train)
    
        ### Use the model generated to predict for training data
        predicted_training_y = clf.predict(X_word_vect)
    
        train_p = metrics.precision_score(y_train, predicted_training_y)
        train_r = metrics.recall_score(y_train, predicted_training_y)
        train_f1= metrics.f1_score(y_train, predicted_training_y)
        training_performance_table[depth, fold, 0]=depth
        training_performance_table[depth, fold, 1]=train_p
        training_performance_table[depth, fold, 2]=train_r
        training_performance_table[depth, fold, 3]=train_f1
        print("Max depth is ", depth, "Prediction Performance for Training Data f1:", train_f1)

        ### Use the model generated to predict for testint data
        x_test_word_vect = count_vect.transform(x_test)
        predicted_testing_y = clf.predict(x_test_word_vect)
    
        test_p= metrics.precision_score(y_test, predicted_testing_y)
        test_r= metrics.recall_score(y_test, predicted_testing_y)
        test_f1 = metrics.f1_score(y_test, predicted_testing_y)
        testing_performance_table[depth, fold, 0]=depth
        testing_performance_table[depth, fold, 1]=test_p
        testing_performance_table[depth, fold, 2]= test_r
        testing_performance_table[depth, fold, 3]= test_f1
        print("Prediction Performance for Testing Data f1:", test_f1)
    
        fold=fold+1
    


### Calculate Average, Minimum, and Maximum f1 score across folds 
The first dimension of training_performance_table refers to a specific max_depth value.
The second dimension of training_performance_table refers to the number of a fold (in a k-fold).
The third dimension of training_performance_table saves f1 score in index "3".

For example, the code below returns an array of f1_score for all k folds for max_depth set to 7.

In [ ]:
f1_array=training_performance_table[7, 1:k+1:1, 3]

#### Numpy offers a convenient way to calculate the average, the minimum, and the maximum value of a given array:
    np.average returns the average of a given array
    np.amin returns the minimum value of a given array
    np.amax regturns the maximum value of a given array

In [ ]:
np.average(f1_array)

In [ ]:
np.amin(training_performance_table[7, 1:k+1:1, 3])

In [ ]:
np.amax(training_performance_table[7, 1:k+1:1, 3])

### We want to calculate the average performance across all folds for each max_depth so that we can compare them. 

In [ ]:
cv_training_f1_summary = np.empty( [high_max_depth+1, 3])
cv_testing_f1_summary = np.empty( [high_max_depth+1, 3])
first_fold = 1
last_fold = 5+1
for depth in range(low_max_depth, high_max_depth+1):
    train_f1_array = training_performance_table[depth, first_fold:last_fold:1 , 3]
    cv_training_f1_summary[depth, 0] = np.average(train_f1_array)
    cv_training_f1_summary[depth, 1] = np.amin(train_f1_array)
    cv_training_f1_summary[depth, 2] = np.amax(train_f1_array)
    test_f1_array = testing_performance_table[depth, first_fold:last_fold:1, 3]
    cv_testing_f1_summary[depth, 0] = np.average(test_f1_array)
    cv_testing_f1_summary[depth, 1] = np.amin(test_f1_array)
    cv_testing_f1_summary[depth, 2] = np.amax(test_f1_array)

### We want to plot the average of f1 scores (across all k folds) for each max_depth, and compare the average of f1 scores for training data and testing data

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import *

In [ ]:
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning )

In [ ]:
depth=np.linspace(low_max_depth, high_max_depth, high_max_depth-low_max_depth+1)
print(depth)

In [ ]:
avg_f1_training = cv_training_f1_summary[low_max_depth:high_max_depth+1, 0]
min_f1_training = cv_training_f1_summary[low_max_depth:high_max_depth+1, 1]
avg_f1_testing = cv_testing_f1_summary[low_max_depth:high_max_depth+1, 0]
min_f1_testing = cv_testing_f1_summary[low_max_depth:high_max_depth+1, 1]

### The following code is for plotting average f1 (across k folds)

In [ ]:
plt.plot(depth, avg_f1_training, label="? fold CV Avg f1 for training")
plt.plot(depth, avg_f1_testing, label="? fold CV Avg f1 for testing")
plt.xlabel('max_depth')
plt.title('? Fold CV Avg f1 vs max_depth for trainiong data and testing data')
plt.legend()
plt.show()

In [ ]:
plt.plot(depth, avg_f1_training, label="? fold CV Avg f1 for training")
plt.plot(depth, min_f1_training, label="? fold CV Min f1 for training")
plt.plot(depth, avg_f1_testing, label="? fold CV Avg f1 for testing")
plt.plot(depth, min_f1_testing, label="? fold CV Min f1 for testing")
plt.xlabel('max_depth')
plt.title('? Fold CV Avg and Min f1 vs max_depth for trainiong data and testing data')
plt.legend()
plt.show()